# Visão Geral das Estruturas de Dados do Pandas
Nesta seção, discutiremos as classes `Series`, `Index` e `DataFrame`. Para isso, leremos um trecho do arquivo CSV com o qual trabalharemos mais tarde. Não se preocupe com essa parte ainda.

## Sobre os Dados
Neste notebook, estaremos trabalhando com 5 linhas dos dados de terremotos coletados entre 18 de setembro de 2018 - 13 de outubro de 2018 (obtidos do US Geological Survey (USGS) usando a [API do USGS](https://earthquake.usgs.gov/fdsnws/event/1/))

## Trabalhando com Arrays NumPy
Vamos ler um pequeno arquivo CSV (usando `numpy`) para alguns dados de amostra.

In [1]:
import numpy as np

data = np.genfromtxt(
    'data/example_data.csv', delimiter=';', 
    names=True, dtype=None, encoding='UTF'
)
data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i8')])

Podemos encontrar as dimensões com o atributo `shape`:

In [2]:
data.shape

(5,)

Podemos encontrar os tipos de dados com o atributo `dtype`:

In [3]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i8')])

Cada uma das entradas no array é uma linha do arquivo CSV. Arrays do NumPy contêm um único tipo de dado (diferente de listas, que permitem tipos mistos); isso possibilita operações rápidas e vetorizadas. Quando lemos os dados, obtivemos um array de objetos `numpy.void`, que são criados para armazenar tipos flexíveis. Isso ocorre porque o NumPy precisa armazenar vários tipos de dados diferentes por linha: quatro strings, um float e um inteiro. Isso significa que não podemos aproveitar as melhorias de desempenho que o NumPy oferece para objetos de um único tipo de dado.

Digamos que queremos encontrar a magnitude máxima&mdash;podemos usar uma **[list comprehension](https://www.python.org/dev/peps/pep-0202/)** para selecionar o terceiro índice de cada linha, que é representado como um objeto `numpy.void`. Isso cria uma lista, o que significa que podemos obter o valor máximo usando a função `max()`:

In [11]:
max([row[3] for row in data])

6.7

Se, em vez disso, criarmos um array do NumPy para cada coluna, essa operação será muito mais fácil (e mais eficiente) de realizar. Podemos usar uma **[dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)** para criar um dicionário onde as chaves são os nomes das colunas e os valores são arrays do NumPy com os dados:

In [5]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'tsunami': array([1, 0, 0, 0, 1])}

Obter a magnitude máxima agora é simplesmente uma questão de selecionar a chave `mag` e chamar o método `max()`. Isso é quase duas vezes mais rápido do que a implementação com list comprehension quando lidamos com apenas 5 entradas; imagine o quão pior a primeira tentativa irá se comportar em grandes conjuntos de dados:

In [10]:
array_dict['mag'].max()

6.7

No entanto, essa representação tem outros problemas. Digamos que queremos obter todas as informações sobre o terremoto com a magnitude máxima, como faríamos isso? Precisaríamos encontrar o índice do valor máximo e, em seguida, para cada uma das chaves no dicionário, pegar esse índice:

In [12]:
np.array([
    value[array_dict['mag'].argmax()] 
    for key, value in array_dict.items()
])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U32')

O resultado agora é um array do NumPy de strings (nossos valores numéricos foram convertidos) e estamos agora no formato anterior. Além disso, considere tentar ordenar os dados pela magnitude do menor para o maior. Na primeira representação, teríamos que ordenar as linhas examinando o 3º índice. Com a segunda representação, teríamos que determinar a ordem dos índices da coluna `mag` e, em seguida, ordenar todos os outros arrays com esses mesmos índices. Claramente, trabalhar com vários arrays do NumPy de diferentes tipos de dados ao mesmo tempo é um pouco complicado. No entanto, o `pandas` constrói em cima dos arrays do NumPy para facilitar isso. Vamos começar nossa exploração do `pandas` com uma visão geral das estruturas de dados.

## `Series`
A classe `Series` fornece uma estrutura de dados para arrays de um único tipo com algumas funcionalidades adicionais.

In [17]:
import pandas as pd

lugares = pd.Series(array_dict['place'], name='place')
lugares

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

Aqui estão alguns atributos comumente usados com objetos `Series`:

| Atributo | Retorna |
| --- | --- |
| `name` | O nome do objeto `Series` |
| `dtype` | O tipo de dado do objeto `Series` |
| `shape` | Dimensões do objeto `Series` em uma tupla na forma `(número de linhas,)` |
| `index` | O objeto `Index` que faz parte do objeto `Series` |
| `values` | Os dados no objeto `Series` |

Na maioria das vezes, objetos do `pandas` usam arrays do NumPy para suas representações internas de dados. No entanto, para alguns tipos de dados, o `pandas` constrói em cima do NumPy para criar seus próprios [arrays](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html). Por essa razão, dependendo do tipo de dado, `values` pode ser um objeto `pandas.array` ou `numpy.array`. Portanto, se precisarmos garantir que recebemos um tipo específico, é recomendado usar o atributo `array` ou o método `to_numpy()`, respectivamente, em vez de `values`.

Agora, vamos ver alguns exemplos usando esses atributos.

### Obtendo o nome da série
O array do NumPy armazenava o nome dos dados no atributo `dtype`; aqui, podemos acessá-lo diretamente:

In [18]:
lugares.name

'place'

### Obtendo o tipo de dado
Um objeto `Series` mantém um único tipo de dado. Aqui, ele é `'O'` para objeto.

In [19]:
lugares.dtype

dtype('O')

### Obtendo as dimensões da série
Assim como no NumPy, podemos usar `shape` para obter as dimensões como `(linhas, colunas)`. Objetos `Series` são uma única coluna, então eles têm valores apenas para a dimensão de linhas.

In [20]:
lugares.shape

(5,)

### Isolando os valores da série
Este objeto `Series` está armazenando seus valores como um array do NumPy:

In [21]:
lugares.values

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype=object)

## `Index`
A adição da classe `Index` torna a classe `Series` mais poderosa do que um array do NumPy. Podemos obter o índice a partir do atributo `index` de um objeto `Series`:

In [22]:
place_index = lugares.index
place_index

RangeIndex(start=0, stop=5, step=1)

Assim como nos objetos `Series`, podemos acessar os dados subjacentes através do atributo `values`. Note que este objeto `Index` também é construído sobre um array do NumPy:

In [23]:
place_index.values

array([0, 1, 2, 3, 4])

Aqui estão alguns atributos comumente usados com objetos `Index`:

| Atributo | Retorna |
| --- | --- |
| `name` | O nome do objeto `Index` |
| `dtype` | O tipo de dado do objeto `Index` |
| `shape` | Dimensões do objeto `Index` |
| `values` | Os dados no objeto `Index` |
| `is_unique` | Verifica se o objeto `Index` tem todos os valores únicos |

Podemos verificar o tipo dos dados subjacentes, da mesma forma que com um objeto `Series`:

In [24]:
place_index.dtype

dtype('int64')

O mesmo vale para as dimensões:

In [25]:
place_index.shape

(5,)

Podemos verificar se os valores são únicos:

In [26]:
place_index.is_unique

True

Com NumPy, podemos realizar operações aritméticas elemento a elemento entre arrays:

In [27]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

array([0, 1, 2])

O Pandas também suporta isso, e o índice determina como as operações elemento a elemento são realizadas. Com a adição, apenas os índices correspondentes são somados:

In [28]:
numbers = np.linspace(0, 10, num=5) # makes numpy array([0, 2.5, 5, 7.5, 10])
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

0     NaN
1     2.5
2     7.5
3    12.5
4    17.5
5     NaN
dtype: float64

## `DataFrame`
Ter um objeto `Series` para cada coluna é uma melhoria em relação à representação do NumPy; no entanto, ainda temos o mesmo problema ao querer ordenar com base em um valor ou pegar uma linha inteira. O `DataFrame` nos dá uma representação de uma tabela formada por vários objetos `Series` que formam as colunas e um objeto `Index` compartilhado que rotula as linhas. Podemos criar um objeto `DataFrame` a partir de qualquer uma das representações do NumPy com as quais estávamos trabalhando anteriormente (também poderíamos criar um objeto `Series` para cada coluna, mas não há necessidade de fazê-lo):

In [29]:
df = pd.DataFrame(array_dict) 

# this will also work with the first representation
# df = pd.DataFrame(data)

df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


Podemos verificar o tipo dos dados subjacentes com `dtypes` (observe que não é `dtype`, como nos objetos `Series` e `Index`, já que cada coluna terá seu próprio tipo de dado):

In [30]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int64
dtype: object

Podemos obter os dados subjacentes com o atributo `values`. Note que isso se parece muito com nossa representação inicial do NumPy:

In [31]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

Podemos isolar as colunas com o atributo `columns`. Observe que as colunas são na verdade um objeto `Index`, mas em um eixo diferente (as colunas são o índice horizontal enquanto as linhas são o índice vertical).

In [32]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

Aqui estão alguns atributos comumente usados:

| Atributo | Retorna |
| --- | --- |
| `dtypes` | Os tipos de dados de cada coluna |
| `shape` | Dimensões do objeto `DataFrame` em uma tupla na forma `(número de linhas, número de colunas)` |
| `index` | O objeto `Index` ao longo das linhas do objeto `DataFrame` |
| `columns` | O nome das colunas (como um objeto `Index`) |
| `values` | Os dados no objeto `DataFrame` |
| `empty` | Verifica se o objeto `DataFrame` está vazio |

O objeto `Index` ao longo das linhas do dataframe pode ser acessado através do atributo `index` (assim como nos objetos `Series`):

In [33]:
df.index

RangeIndex(start=0, stop=5, step=1)

Assim como nos objetos `Series` e `Index`, podemos obter as dimensões do dataframe com o atributo `shape`. O resultado é da forma `(número de linhas, número de colunas)`. Nosso dataframe tem 5 linhas e 6 colunas:

In [34]:
df.shape

(5, 6)

Note que também podemos realizar operações aritméticas em dataframes. O Pandas realizará a operação apenas quando tanto o índice quanto a coluna coincidirem. Aqui, demonstramos a adição. Como a adição com strings significa concatenação, o Pandas concatenou as colunas de strings (`time`, `place`, `magType` e `alert`) entre os dataframes. As colunas numéricas (`mag` e `tsunami`) foram somadas:

In [35]:
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


<hr>
<div>
    <a href="../ch_01/introduction_to_data_analysis.ipynb">
        <button style="float: left;">&#8592; Chapter 1</button>
    </a>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>